In [3]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tinydb import TinyDB, Query

In [97]:

db = TinyDB('/home/thomas/Python Projects/hwh/watches/db.json')

User = Query()

In [4]:
url = "http://www.chrono24.com/alangesoehne/datograph-perp-cal-chrono-pt-new--id5645635.htm"
url = "https://www.chrono24.com//sinn/ezm-1-with-3-h-dial--id9313636.htm"

In [5]:
driver = webdriver.Firefox()
driver.get(url)

source = driver.page_source
driver.close()

In [56]:
brandurl = "https://www.chrono24.com/rolex/index.htm?man=rolex&pageSize=120&showpage=1"
driver = webdriver.Firefox()
driver.get(url)

brandsource = driver.page_source
driver.close()



In [6]:
def getbrandlinks(brand, soupbrandsource):
    divs = soupbrandsource.find_all('a')
    hrefs = [j.get('href') for j in divs if j is not None and brand.lower() in str(j) and 'index.htm' not in str(j) and '--id' in str(j)]
    return hrefs

In [7]:
def getsourcefromurl(url,driver,closewebdriver=False):
    #driver = webdriver.Firefox()
    driver.get(url)

    source = driver.page_source
    if closewebdriver:
        driver.close()
        
    return BeautifulSoup(source, 'lxml')


In [18]:
def parsedata(soupsource):

    elem = soupsource.find('section',{'class':'specifications p-y-5'}).table

    rows = elem.find_all('tr')

    title = ''

    n = 0
    
    result = {}

    pricebox = soupsource.find('span',{'class':'d-block'}).span
    if pricebox.text.lower() != 'price on request':
        print(pricebox.text)
        currency = pricebox.span.text
        price = pricebox.text.replace('{currency}'.format(currency=currency),'').replace(',','')
        result['Price'] = price
        result['PriceCurrency'] = currency
    
    for nnr, row in enumerate(rows):
        if len(row.find_all('td')) == 1:
            title = row.td.text.strip()
            n = -1
        else:
            tmp = row.find_all('td')
            subtitle = tmp[0].text.strip()
            if subtitle == 'Condition':
                value = tmp[1].a.text.strip()
            else:
                value = tmp[1].text.strip()
        n += 1   
        if n > 0:
            result[subtitle] = value
    
    return result

    

In [19]:
parsedata(BeautifulSoup(source))

/home/thomas/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/thomas/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


{'Bracelet material': 'Leather',
 'Brand': 'Sinn',
 'Case material': 'Titanium',
 'Condition': 'Good',
 'Gender': "Men's watch/Unisex",
 'Listing number': '5jmg49',
 'Location': 'United Kingdom, London',
 'Movement': 'Automatic',
 'Reference number': '503EZM1',
 'Scope of delivery': 'Original box, no original papers',
 'Year': '1998'}

In [ ]:
#### MAIN ####




rooturl = 'https://www.chrono24.com'
brand = 'rolex'
brandurl = rooturl + "/{brand}/index.htm?man={brand}&pageSize=120&showpage=1".format(brand=brand)

print(brandurl)

driver = webdriver.Firefox()
brandsource = getsourcefromurl(brandurl,driver,False)
brandpages = getbrandlinks(brand,brandsource)

results = []

for page in brandpages[:100]:
    if len(db.search(User.page==page)) == 0:
        print(page)
        full_url = rooturl + page
        watchsource = getsourcefromurl(full_url,driver,False)
        result = parsedata(watchsource)
        result['page'] = page
        db.insert(result)
        #results.append(result)


driver.close()


https://www.chrono24.com/rolex/index.htm?man=rolex&pageSize=120&showpage=1
/rolex/datejust-41mm-steel--18k-rose-gold-chocolate-dial-126331--id5473058.htm
/rolex/sky-dweller-42mm-stainless-steel--18k-yellow-gold-326933--id7572898.htm
/rolex/daytona-steel--18k-yellow-gold-white-dial-watch-116503--id3346912.htm
/rolex/day-date-36mm-18k-yellow-gold-president-diamond-dial--118238--id3346770.htm
/rolex/datejust-ii-41mm-18k-white-gold-bezel-blue-roman-dial-116334--id7595264.htm
/rolex/datejust-41mm-steel--18k-yellow-gold-jubilee-bracelet-12633--id5472953.htm
/rolex/datejust-41-steel-18k-rose-gold-sundust-diamond-dial-126331--id5472994.htm
/rolex/sky-dweller-42mm-18k-yellow-gold--black-dial-watch-326938--id8634456.htm
/rolex/sea-dweller-deepsea-d-blue-steel-44mm---126660--id8439624.htm
/rolex/datejust--id9091167.htm
/rolex/sky-dweller-326933--id9355165.htm
/rolex/gmt-master-ii--id9343048.htm
/rolex/day-date-40mm-18k-yellow-gold-president-diamond-dial--228238--id4714036.htm
/rolex/sea-dweller-d

/home/thomas/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/thomas/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


AttributeError: 'NoneType' object has no attribute 'text'

In [106]:
#db.search(User.page == '/rolex/gmt-master-ii-18k-yellow-gold-with-factory-diamonds-116758sa--id7565951.htm')


out = db.search(User.Brand == 'Rolex')
pd.DataFrame(out)


,Bezel material,Bracelet color,Bracelet material,Brand,Case diameter,Case material,Clasp,Clasp material,Code,Condition,...,Listing number,Location,Model,Movement,Price,PriceCurrency,Reference number,Scope of delivery,Water resistance,page
0,Yellow gold,Gold,Yellow gold,Rolex,40 mm,Yellow gold,Fold clasp,Yellow gold,WTCH05928,Very good,...,4i5xb5,"United States of America, New York, New York",GMT-Master II,Automatic,99999,$,116758SANR,"Original box, original papers",10 ATM,/rolex/gmt-master-ii-18k-yellow-gold-with-fact...
1,Ceramic,Gold/Steel,Gold/Steel,Rolex,40 mm,Gold/Steel,Fold clasp,Gold/Steel,WTCH0000,Unworn,...,50dvy5,"United States of America, New York, New York",Submariner Date,Automatic,13499,$,116613,"Original box, original papers",30 ATM,/rolex/submariner-steel--18k-yellow-gold-blue-...
